In [ ]:
!pip -q install -U huey

import os
import time
import json
import random
import threading
from datetime import datetime

from huey import SqliteHuey, crontab
from huey.constants import WORKER_THREAD

DB_PATH = "/content/huey_demo.db"
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)

huey = SqliteHuey(
    name="colab-huey",
    filename=DB_PATH,
    results=True,
    store_none=False,
    utc=True,
)

print("Huey backend:", type(huey).__name__)
print("SQLite DB at:", DB_PATH)

In [ ]:
EVENT_LOG = []

@huey.signal()
def _log_all_signals(signal, task, exc=None):
    EVENT_LOG.append({
        "ts": datetime.utcnow().isoformat() + "Z",
        "signal": str(signal),
        "task": getattr(task, "name", None),
        "id": getattr(task, "id", None),
        "args": getattr(task, "args", None),
        "kwargs": getattr(task, "kwargs", None),
        "exc": repr(exc) if exc else None,
    })

def print_latest_events(n=10):
    print("\n--- Latest Huey events ---")
    for row in EVENT_LOG[-n:]:
        print(json.dumps(row, indent=2))

In [ ]:
@huey.task(priority=50)
def quick_add(a, b):
    return a + b

@huey.task(priority=10)
def slow_io(seconds=1.0):
    time.sleep(seconds)
    return f"slept={seconds}"

@huey.task(retries=3, retry_delay=1, priority=100)
def flaky_network_call(p_fail=0.6):
    if random.random() < p_fail:
        raise RuntimeError("Transient failure (simulated)")
    return "OK"

@huey.task(context=True, priority=60)
def cpu_pi_estimate(samples=200_000, task=None):
    inside = 0
    rnd = random.random
    for _ in range(samples):
        x, y = rnd(), rnd()
        if x*x + y*y <= 1.0:
            inside += 1
    est = 4.0 * inside / samples
    return {"task_id": task.id if task else None, "pi_estimate": est, "samples": samples}

In [ ]:
@huey.lock_task("demo:daily-sync")
@huey.task()
def locked_sync_job(tag="sync"):
    time.sleep(1.0)
    return f"locked-job-done:{tag}:{datetime.utcnow().isoformat()}Z"

@huey.task()
def fetch_number(seed=7):
    random.seed(seed)
    return random.randint(1, 100)

@huey.task()
def transform_number(x, scale=3):
    return x * scale

@huey.task()
def store_result(x):
    return {"stored_value": x, "stored_at": datetime.utcnow().isoformat() + "Z"}

In [ ]:
TICK = {"count": 0}

@huey.task()
def heartbeat():
    TICK["count"] += 1
    print(f"[heartbeat] tick={TICK['count']} utc={datetime.utcnow().isoformat()}Z")

@huey.periodic_task(crontab(minute="*"))
def heartbeat_minutely():
    heartbeat()

_TIMER_STATE = {"running": False, "timer": None}

def start_seconds_heartbeat(interval_sec=15):
    _TIMER_STATE["running"] = True
    def _tick():
        if not _TIMER_STATE["running"]:
            return
        huey.enqueue(heartbeat.s())
        t = threading.Timer(interval_sec, _tick)
        _TIMER_STATE["timer"] = t
        t.start()
    _tick()

def stop_seconds_heartbeat():
    _TIMER_STATE["running"] = False
    t = _TIMER_STATE.get("timer")
    if t is not None:
        try:
            t.cancel()
        except Exception:
            pass
    _TIMER_STATE["timer"] = None

In [2]:
consumer = huey.create_consumer(
    workers=4,
    worker_type=WORKER_THREAD,
    periodic=True,
    initial_delay=0.1,
    backoff=1.15,
    max_delay=2.0,
    scheduler_interval=1,
    check_worker_health=True,
    health_check_interval=10,
    flush_locks=False,
)

consumer_thread = threading.Thread(target=consumer.run, daemon=True)
consumer_thread.start()
print("Consumer started (threaded).")

print("\nEnqueue basics...")
r1 = quick_add(10, 32)
r2 = slow_io(0.75)
print("quick_add result:", r1(blocking=True, timeout=5))
print("slow_io result:", r2(blocking=True, timeout=5))

print("\nRetries + priority demo (flaky task)...")
rf = flaky_network_call(p_fail=0.7)
try:
    print("flaky_network_call result:", rf(blocking=True, timeout=10))
except Exception as e:
    print("flaky_network_call failed even after retries:", repr(e))

print("\nContext task (task id inside payload)...")
rp = cpu_pi_estimate(samples=150_000)
print("pi payload:", rp(blocking=True, timeout=20))

print("\nLocks demo: enqueue multiple locked jobs quickly (should serialize)...")
locked_results = [locked_sync_job(tag=f"run{i}") for i in range(3)]
print([res(blocking=True, timeout=10) for res in locked_results])

print("\nScheduling demo: run slow_io in ~3 seconds...")
rs = slow_io.schedule(args=(0.25,), delay=3)
print("scheduled handle:", rs)
print("scheduled slow_io result:", rs(blocking=True, timeout=10))

print("\nRevoke demo: schedule a task in 5s then revoke before it runs...")
rv = slow_io.schedule(args=(0.1,), delay=5)
rv.revoke()
time.sleep(6)
try:
    out = rv(blocking=False)
    print("revoked task output:", out)
except Exception as e:
    print("revoked task did not produce result (expected):", type(e).__name__, str(e)[:120])

print("\nPipeline demo...")
pipeline = (
    fetch_number.s(123)
    .then(transform_number, 5)
    .then(store_result)
)
pipe_res = huey.enqueue(pipeline)
print("pipeline final result:", pipe_res(blocking=True, timeout=10))

print("\nStarting 15-second heartbeat demo for ~40 seconds...")
start_seconds_heartbeat(interval_sec=15)
time.sleep(40)
stop_seconds_heartbeat()
print("Stopped 15-second heartbeat demo.")

print_latest_events(12)

print("\nStopping consumer gracefully...")
consumer.stop(graceful=True)
consumer_thread.join(timeout=5)
print("Consumer stopped.")

/tmp/ipython-input-2876153222.py:51: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "ts": datetime.utcnow().isoformat() + "Z",
Exception in thread Thread-3 (run):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/huey/consumer.py", line 437, in run
    self.start()
  File "/usr/local/lib/python3.12/dist-packages/huey/consumer.py", line 409, in start
    self._set_signal_handlers()
  File "/usr/local/lib/python3.12/dist-packages/huey/consumer.py", line 513, in _set_signal_handlers
    signal.signal(signal.SIGTERM, self._handle_stop_signal)
  File "/usr/lib/python3.12/signal.py", line 58, in signal
    handl

Huey backend: SqliteHuey
SQLite DB at: /content/huey_demo.db
Consumer started (threaded).

Enqueue basics...
[heartbeat] tick=1 utc=2026-02-21T13:56:47.907104Z
quick_add result: 42


/tmp/ipython-input-2876153222.py:128: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[heartbeat] tick={TICK['count']} utc={datetime.utcnow().isoformat()}Z")


slow_io result: slept=0.75

Retries + priority demo (flaky task)...
flaky_network_call result: OK

Context task (task id inside payload)...
pi payload: {'task_id': '67d28cc8-f46a-45b2-b70e-8609b13121ef', 'pi_estimate': 3.1431466666666665, 'samples': 150000}

Locks demo: enqueue multiple locked jobs quickly (should serialize)...


/tmp/ipython-input-2876153222.py:101: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return f"locked-job-done:{tag}:{datetime.utcnow().isoformat()}Z"


['locked-job-done:run0:2026-02-21T13:56:49.909515Z', 'locked-job-done:run1:2026-02-21T13:56:49.971290Z', 'locked-job-done:run2:2026-02-21T13:56:50.007559Z']

Scheduling demo: run slow_io in ~3 seconds...
scheduled handle: <Result: task 324d6d05-2dc0-4233-9760-26d5d5a39b0b>
scheduled slow_io result: slept=0.25

Revoke demo: schedule a task in 5s then revoke before it runs...


revoked task output: None

Pipeline demo...
pipeline final result: [7, 35, {'stored_value': 35, 'stored_at': '2026-02-21T13:57:00.901327Z'}]

Starting 15-second heartbeat demo for ~40 seconds...
[heartbeat] tick=2 utc=2026-02-21T13:57:01.002315Z


/tmp/ipython-input-2876153222.py:117: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return {"stored_value": x, "stored_at": datetime.utcnow().isoformat() + "Z"}


[heartbeat] tick=3 utc=2026-02-21T13:57:15.987036Z
[heartbeat] tick=4 utc=2026-02-21T13:57:31.673487Z
Stopped 15-second heartbeat demo.

--- Latest Huey events ---
{
  "ts": "2026-02-21T13:57:00.901008Z",
  "signal": "enqueued",
  "task": "store_result",
  "id": "f083738e-fffe-4693-abd0-5f85abaf8dec",
  "args": [
    35
  ],
  "kwargs": {},
  "exc": null
}
{
  "ts": "2026-02-21T13:57:00.901182Z",
  "signal": "executing",
  "task": "store_result",
  "id": "f083738e-fffe-4693-abd0-5f85abaf8dec",
  "args": [
    35
  ],
  "kwargs": {},
  "exc": null
}
{
  "ts": "2026-02-21T13:57:00.901752Z",
  "signal": "complete",
  "task": "store_result",
  "id": "f083738e-fffe-4693-abd0-5f85abaf8dec",
  "args": [
    35
  ],
  "kwargs": {},
  "exc": null
}
{
  "ts": "2026-02-21T13:57:00.962195Z",
  "signal": "enqueued",
  "task": "heartbeat",
  "id": "a4d0244a-d549-4ad4-bb66-3f3f3ee38b99",
  "args": [],
  "kwargs": {},
  "exc": null
}
{
  "ts": "2026-02-21T13:57:01.002283Z",
  "signal": "executing",
  